In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd

from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score



In [2]:
from lore_sa.util import record2str, neuclidean

In [3]:
from lore_sa.dataset import Dataset

In [4]:
dataset = Dataset.from_csv('test/resources/adult.csv')
dataset.df.dropna(inplace = True)
dataset.df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [5]:
dataset.df.drop(['fnlwgt', 'education-num'], inplace=True, axis=1)
dataset.df

,age,workclass,education,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,Assoc-acdm,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,HS-grad,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,HS-grad,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [8]:
from lore_sa.encoder_decoder import OneHotEnc,TargetEnc
encoder = OneHotEnc()

dataset_encoded = encoder.encode(dataset,['race','sex'])
dataset_encoded

,age,workclass,education,marital-status,occupation,relationship,capital-gain,capital-loss,hours-per-week,native-country,class,race=Amer-Indian-Eskimo,race=Asian-Pac-Islander,race=Black,race=Other,race=White,sex=Female,sex=Male
0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,2174,0,40,United-States,<=50K,0,0,0,0,1,0,1
1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,0,0,13,United-States,<=50K,0,0,0,0,1,0,1
2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,0,0,40,United-States,<=50K,0,0,0,0,1,0,1
3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,0,0,40,United-States,<=50K,0,0,1,0,0,0,1
4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,0,0,40,Cuba,<=50K,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,Assoc-acdm,Married-civ-spouse,Tech-support,Wife,0,0,38,United-States,<=50K,0,0,0,0,1,1,0
32557,40,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,0,0,40,United-States,>50K,0,0,0,0,1,0,1
32558,58,Private,HS-grad,Widowed,Adm-clerical,Unmarried,0,0,40,United-States,<=50K,0,0,0,0,1,1,0
32559,22,Private,HS-grad,Never-married,Adm-clerical,Own-child,0,0,20,United-States,<=50K,0,0,0,0,1,0,1


In [9]:
dataset_encoded.size


542916

In [10]:
dataset.df.size

542916

In [ ]:
encoder = MyTargetEnc()

In [32]:
#dataset encoded only for the user-specified features
dataset.class_name = "capital-gain"
df_enc = encoder.encode(dataset,['workclass','education','occupation','race','sex'], target = "capital-gain")
df_enc

,age,marital-status,relationship,capital-gain,capital-loss,hours-per-week,native-country,class,age,workclass,...,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,Never-married,Not-in-family,2174,0,40,United-States,<=50K,39,684.306489,...,Never-married,497.097286,Not-in-family,1142.651024,1342.476546,2174,0,40,United-States,<=50K
1,50,Married-civ-spouse,Husband,0,0,13,United-States,<=50K,50,1913.134454,...,Married-civ-spouse,2185.330912,Husband,1142.651024,1342.476546,0,0,13,United-States,<=50K
2,38,Divorced,Not-in-family,0,0,40,United-States,<=50K,38,879.858207,...,Divorced,258.140000,Not-in-family,1142.651024,1342.476546,0,0,40,United-States,<=50K
3,53,Married-civ-spouse,Husband,0,0,40,United-States,<=50K,53,879.858207,...,Married-civ-spouse,258.140000,Husband,604.722045,1342.476546,0,0,40,United-States,<=50K
4,28,Married-civ-spouse,Wife,0,0,40,Cuba,<=50K,28,879.858207,...,Married-civ-spouse,2717.880139,Wife,604.722045,570.176753,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Married-civ-spouse,Wife,0,0,38,United-States,<=50K,27,879.858207,...,Married-civ-spouse,668.105263,Wife,1142.651024,570.176753,0,0,38,United-States,<=50K
32557,40,Married-civ-spouse,Husband,0,0,40,United-States,>50K,40,879.858207,...,Married-civ-spouse,323.825025,Husband,1142.651024,1342.476546,0,0,40,United-States,>50K
32558,58,Widowed,Unmarried,0,0,40,United-States,<=50K,58,879.858207,...,Widowed,497.097286,Unmarried,1142.651024,570.176753,0,0,40,United-States,<=50K
32559,22,Never-married,Own-child,0,0,20,United-States,<=50K,22,879.858207,...,Never-married,497.097286,Own-child,1142.651024,1342.476546,0,0,20,United-States,<=50K


In [33]:
print (encoder)

#dataset back in his original format
encoder.decode(dataset)

TargetEncoder - original features encoded: workclass,education,occupation,race,sex


,age,workclass,education,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,Assoc-acdm,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,HS-grad,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,HS-grad,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K
